In [40]:
import pandas as pd

# Load your real dataset
df = pd.read_csv('../data/processed/full_similarity_with_returns.csv')

# Preview the data
print(df.head())

df = df.rename(columns={"filing_date": "month"})
df = df.rename(columns={"similarity": "similarity_score"})


df


    cik  tic       month  similarity      ret
0  1750  air  1994-01-31         NaN  14.6207
1  1750  air  1994-04-30    0.991912      NaN
2  1750  air  1994-08-31    0.934598  -2.7027
3  1750  air  1994-10-31    0.928699  -4.8462
4  1750  air  1995-01-31    0.868299   1.8318


,cik,tic,month,similarity_score,ret
0,1750,air,1994-01-31,NaN,14.6207
1,1750,air,1994-04-30,0.991912,NaN
2,1750,air,1994-08-31,0.934598,-2.7027
3,1750,air,1994-10-31,0.928699,-4.8462
4,1750,air,1995-01-31,0.868299,1.8318
...,...,...,...,...,...
1303798,2029303,NaN,2024-11-30,NaN,NaN
1303799,2031561,NaN,2024-11-30,NaN,NaN
1303800,2032950,NaN,2024-12-31,NaN,NaN
1303801,2033615,NaN,2024-12-31,NaN,NaN


### Removing missing values

It is valid to drop missing rows. Need description.

In [41]:
print(df.isna().sum())
print(df.shape)

df = df.dropna(subset=['similarity_score'])
df = df.dropna(subset=['tic'])
df = df.dropna(subset=['ret'])

print(df.isna().sum())
print(df.shape)

df

cik                       0
tic                  956934
month                     0
similarity_score    1004951
ret                 1152829
dtype: int64
(1303803, 5)
cik                 0
tic                 0
month               0
similarity_score    0
ret                 0
dtype: int64
(34027, 5)


,cik,tic,month,similarity_score,ret
2,1750,air,1994-08-31,0.934598,-2.7027
3,1750,air,1994-10-31,0.928699,-4.8462
4,1750,air,1995-01-31,0.868299,1.8318
6,1800,abt,1994-05-31,0.934639,5.2863
7,1800,abt,1994-08-31,0.888515,6.6667
...,...,...,...,...,...
1299053,85535,rgld,2024-12-31,0.923907,-9.8523
1300115,1058811,immr,2024-12-31,0.985401,-2.3490
1300909,1393612,dfs,2024-12-31,0.977140,-5.0430
1300910,1393612,dfs,2024-12-31,0.998017,-5.0430


We also need to remove rows where there is only one row per month, since we can't construct that portfolio.

In [43]:
# Step 1: Remove months with only 1 stock
month_counts = df['month'].value_counts()
valid_months = month_counts[month_counts > 1].index
df = df[df['month'].isin(valid_months)].copy()

Assigning quintiles

In [ ]:
# Assign quintiles within each month using similarity_score
def assign_quintile(x):
    if len(x) < 5:
        max_signal = x['similarity_score'].max()
        min_signal = x['similarity_score'].min()
        return x['similarity_score'].apply(lambda s: 5 if s == max_signal else (1 if s == min_signal else 3))
    else:
        return pd.qcut(x['similarity_score'], 5, labels=[1,2,3,4,5])

df['quintile'] = df.groupby('month', group_keys=False).apply(assign_quintile).astype(int)

ValueError: Bin edges must be unique: Index([0.8850336239467398, 0.9837652593557836,  0.984786168888206,
       0.9884496934511036, 0.9884496934511036, 0.9953769176599754],
      dtype='float64', name='similarity_score').
You can drop duplicate edges by setting the 'duplicates' kwarg

In [ ]:
df

In [ ]:
#Compute monthly long-short portfolio returns
portfolio_rets = []
for month, group in df.groupby('month'):
    long_ret = group.loc[group['quintile'] == 5, 'ret'].mean()
    short_ret = group.loc[group['quintile'] == 1, 'ret'].mean()
    long_short = long_ret - short_ret
    portfolio_rets.append({'month': month, 'portfolio_return': long_short})

portfolio_df = pd.DataFrame(portfolio_rets)
portfolio_df['month'] = pd.to_datetime(portfolio_df['month'])
portfolio_df = portfolio_df.sort_values('month')

In [ ]:
#Calculate cumulative return (growth of $1, if ret is percent, convert to decimal first)
portfolio_df['portfolio_return_decimal'] = portfolio_df['portfolio_return'] / 100
portfolio_df['cum_portfolio'] = (1 + portfolio_df['portfolio_return_decimal']).cumprod()


In [ ]:
#Output your results
print(portfolio_df.head())
# Save your portfolio DataFrame as a CSV file in the right path
portfolio_df.to_csv("../data/processed/full_portfolio_returns.csv", index=False)


In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.plot(portfolio_df['month'], portfolio_df['portfolio_return'], marker='o', label='Monthly Portfolio Return')
plt.xlabel('Month')
plt.ylabel('Monthly Long-Short Return (%)')
plt.title('Monthly Long-Short Portfolio Return')
plt.legend()
plt.grid(alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [ ]:
# S&P 500
import pandas as pd

# Download S&P 500 from FRED
fred_url = "https://fred.stlouisfed.org/graph/fredgraph.csv?id=SP500"
sp500 = pd.read_csv(fred_url)

# Check real column names
print(sp500.columns)
# Likely: ['observation_date', 'SP500']

# Convert date and rename for consistency
sp500['month'] = pd.to_datetime(sp500['observation_date'])
sp500 = sp500.rename(columns={'SP500':'sp500_index'})

# Filter to your project range
start_date = "1993-03-01"
end_date = "2024-11-30"
sp500 = sp500[(sp500['month'] >= start_date) & (sp500['month'] <= end_date)]

# Calculate monthly returns (in percent)
sp500['sp500_return'] = sp500['sp500_index'].pct_change() * 100

# Format date as string if merging
sp500['month'] = sp500['month'].dt.strftime('%Y-%m-%d')

# Drop first row with NaN return
sp500 = sp500.dropna(subset=['sp500_return'])

print(sp500[['month', 'sp500_return']].head())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Load your saved portfolio results (from CSV)
portfolio_df = pd.read_csv("../data/processed/full_portfolio_returns.csv")
portfolio_df['month'] = pd.to_datetime(portfolio_df['month']).dt.strftime('%Y-%m-%d')

# 2. S&P 500 already loaded and processed as `sp500`
# (from your previous notebook cell using FRED URL)
# sp500['month'] and sp500['sp500_return'] are ready

# 3. Calculate cumulative returns for both
# Portfolio
if 'cum_portfolio' not in portfolio_df.columns:
    portfolio_df['portfolio_return_decimal'] = portfolio_df['portfolio_return'] / 100
    portfolio_df['cum_portfolio'] = (1 + portfolio_df['portfolio_return_decimal']).cumprod()
# S&P 500
sp500['sp500_return_decimal'] = sp500['sp500_return'] / 100
sp500['cum_sp500'] = (1 + sp500['sp500_return_decimal']).cumprod()

# 4. Merge on 'month'
merged = pd.merge(
    portfolio_df[['month', 'cum_portfolio']],
    sp500[['month', 'cum_sp500']],
    on='month', how='inner'
)



# 6. Plot
plt.figure(figsize=(10,5))
plt.plot(merged['month'], merged['cum_portfolio'], label='Long-Short Portfolio (Lazy Price)')
plt.plot(merged['month'], merged['cum_sp500'], label='S&P 500')
plt.xlabel('Month')
plt.ylabel('Cumulative Return (Growth of $1)')
plt.title('Cumulative Performance: Portfolio vs S&P 500')
plt.legend()
plt.grid(alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Logic
1.
a. Sort all firms by their similarity_score for that month (the higher the score, the more “similar” their filing is to the previous one).
b. Assign quintiles (divide into 5 groups) based on the similarity_score within that month:
Top 20% (5th quintile): Firms with the highest similarity
Bottom 20% (1st quintile): Firms with the lowest similarity
2.
Long leg: Go long (buy) all stocks in the top quintile (highest similarity).
Short leg: Go short (sell) all stocks in the bottom quintile (lowest similarity).

Returns are equally weighted within each leg (i.e., average the returns for the long group and the short group).
3.
Long-short return = Mean(return of long leg) - Mean(return of short leg)

Key
If the long-short return is consistently positive:It means firms with high similarity (i.e., "lazy" prices) tend to outperform those with low similarity, after controlling for general market movements.
